In [1]:
import pandas as pd 


In [2]:
sales_df = pd.read_excel('Sales Report Data Cleaning.xlsx')


In [3]:
sales_df['EVENT_USAGE_DATE'] = pd.to_datetime(sales_df['EVENT_USAGE_DATE'], format='%Y-%m-%d', utc=True)
sales_df['EVENT_USAGE_DATE'] = sales_df['EVENT_USAGE_DATE'].dt.strftime('%Y-%m-%d')
sales_df = sales_df[~sales_df['ADJ.PGS'].isin(['C','N','W'])]



In [4]:
attendance_df = pd.read_csv('attendance.csv')
attendance_df = attendance_df[attendance_df['TICKET_ID'].isin(sales_df['TICKET_ID'])]

attendance_df['EVENT_DATE'] = pd.to_datetime(attendance_df['EVENT_DATE'], format='%d-%b-%Y %I:%M %p', utc=True)
attendance_df['EVENT_DATE'] = attendance_df['EVENT_DATE'].dt.strftime('%Y-%m-%d')

/var/folders/23/z3k66lxx0cd1jl6zg3b31qhr0000gn/T/ipykernel_2395/1350474842.py:1: DtypeWarning: Columns (30,46) have mixed types. Specify dtype option on import or set low_memory=False.
  attendance_df = pd.read_csv('attendance.csv')


In [5]:
def categorize_name(condition):
    if 'Single' in str(condition):
        return 'Singles'
    elif 'Group' in str(condition):
        return 'Groups'
    elif 'Package' in str(condition):
        return 'Packages'
    else:
        return 'N/A'


#apply function to condition column
sales_df['Name'] = sales_df['Name'].apply(categorize_name)
sales_df

,TICKET_ID,BUYER_TYPE_DESC,BUYER_TYPE_CODE,PRICE_SCALE_CODE,Name,SEASON_CODE,FINANCIAL_ACCOUNT_ID,FINANCIAL_ACCOUNT_NAME,SERIES_CODE,PACKAGE_DESC,...,F&B_ATTENDANCE,Total Loaded Value,SOLD_COMP,SCANNED_DATE_TIME,SCAN_DEVICE_DESCRIPTION,SCAN_AREA_CODE,SCAN_MEDIA_TYPE_DESCRIPTION,SCAN_MEDIA_TYPE_CODE,SCANNED,Entrance 30M Interval
1,153111110,Full Season Package 4 2023,FSP423,TBRES,Packages,24WOOSOX,1084732.0,3 Media Web,24FULL,Full Season Package,...,CONCESSION,0,Sold,14:59:00,Gate D-Home Plate 01,D-HP,MYTICKETS_MOBILE_TICKET,MM,1,14:30:00
3,153111109,Full Season Package 4 2023,FSP423,TBRES,Packages,24WOOSOX,1084732.0,3 Media Web,24FULL,Full Season Package,...,CONCESSION,0,Sold,15:47:00,Gate D-Home Plate 01,D-HP,MYTICKETS_MOBILE_TICKET,MM,1,15:30:00
5,153111108,Full Season Package 4 2023,FSP423,TBRES,Packages,24WOOSOX,1084732.0,3 Media Web,24FULL,Full Season Package,...,CONCESSION,0,Sold,15:47:00,Gate D-Home Plate 01,D-HP,MYTICKETS_MOBILE_TICKET,MM,1,15:30:00
7,153111107,Full Season Package 4 2023,FSP423,TBRES,Packages,24WOOSOX,1084732.0,3 Media Web,24FULL,Full Season Package,...,CONCESSION,0,Sold,15:47:00,Gate D-Home Plate 01,D-HP,MYTICKETS_MOBILE_TICKET,MM,1,15:30:00
8,158437844,Group Tickets $10.00 LV,GROUPB,FBFB,Groups,24WOOSOX,1351917.0,MOSES,NaN,NaN,...,CONCESSION,10,Sold,16:20:00,Gate D-Home Plate 01,D-HP,UNSPECIFIED_TICKET,US,1,16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566340,171236783,Yaz Tickets DOG,YADOG,GA,Singles,24WOOSOX,NaN,NaN,NaN,NaN,...,CONCESSION,0,Sold,14:35:00,Gate A-Fallon Health Square 05,A-FS,UNSPECIFIED_TICKET,US,1,14:30:00
566341,171236782,Yaz Tickets DOG,YADOG,GA,Singles,24WOOSOX,NaN,NaN,NaN,NaN,...,CONCESSION,0,Sold,14:35:00,Gate A-Fallon Health Square 05,A-FS,UNSPECIFIED_TICKET,US,1,14:30:00
566344,171237027,Yaz Tickets DOG,YADOG,GA,Singles,24WOOSOX,NaN,NaN,NaN,NaN,...,CONCESSION,0,Sold,14:47:00,Gate A-Fallon Health Square 01,A-FS,UNSPECIFIED_TICKET,US,1,14:30:00
566345,171237026,Yaz Tickets DOG,YADOG,GA,Singles,24WOOSOX,NaN,NaN,NaN,NaN,...,CONCESSION,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [6]:
sales_singles_df = sales_df[sales_df['Name'] == 'Singles']
attendance_singles_df = attendance_df[attendance_df['TICKET_ID'].isin(sales_singles_df['TICKET_ID'])]
sales_singles_df = sales_singles_df.groupby('EVENT_USAGE_DATE').size().reset_index(name='Sales_Singles_Count')
attendance_singles_df = attendance_singles_df.groupby('EVENT_DATE').size().reset_index(name='Attendance_Singles_Count')





In [7]:
singles_df = pd.merge(sales_singles_df, attendance_singles_df, how= 'inner', right_on='EVENT_DATE', left_on ='EVENT_USAGE_DATE')
singles_df = singles_df.drop(columns=['EVENT_DATE'])
singles_df


,EVENT_USAGE_DATE,Sales_Singles_Count,Attendance_Singles_Count
0,2024-04-02,2750,1956
1,2024-04-05,590,485
2,2024-04-06,637,510
3,2024-04-07,621,560
4,2024-04-16,1160,1079
...,...,...,...
68,2024-09-11,729,627
69,2024-09-12,979,767
70,2024-09-13,2729,2111
71,2024-09-14,3126,2492


In [8]:
sales_groups_df = sales_df[sales_df['Name'] == 'Groups']
attendance_groups_df = attendance_df[attendance_df['TICKET_ID'].isin(sales_groups_df['TICKET_ID'])]
sales_groups_df = sales_groups_df.groupby('EVENT_USAGE_DATE').size().reset_index(name='Sales_Groups_Count')
attendance_groups_df = attendance_groups_df.groupby('EVENT_DATE').size().reset_index(name='Attendance_Groups_Count')

In [9]:
groups_df = pd.merge(sales_groups_df, attendance_groups_df, how= 'inner', right_on='EVENT_DATE', left_on ='EVENT_USAGE_DATE')
groups_df = groups_df.drop(columns=['EVENT_DATE'])
groups_df


,EVENT_USAGE_DATE,Sales_Groups_Count,Attendance_Groups_Count
0,2024-04-02,847,293
1,2024-04-05,686,359
2,2024-04-06,354,254
3,2024-04-07,446,388
4,2024-04-16,135,84
...,...,...,...
68,2024-09-11,977,534
69,2024-09-12,1821,1013
70,2024-09-13,2484,1437
71,2024-09-14,2151,1255


In [10]:
final_df = pd.merge(singles_df, groups_df, how='inner', on='EVENT_USAGE_DATE')
final_df

,EVENT_USAGE_DATE,Sales_Singles_Count,Attendance_Singles_Count,Sales_Groups_Count,Attendance_Groups_Count
0,2024-04-02,2750,1956,847,293
1,2024-04-05,590,485,686,359
2,2024-04-06,637,510,354,254
3,2024-04-07,621,560,446,388
4,2024-04-16,1160,1079,135,84
...,...,...,...,...,...
68,2024-09-11,729,627,977,534
69,2024-09-12,979,767,1821,1013
70,2024-09-13,2729,2111,2484,1437
71,2024-09-14,3126,2492,2151,1255


In [11]:
sales_packages_df = sales_df[sales_df['Name'] == 'Packages']
attendance_packages_df = attendance_df[attendance_df['TICKET_ID'].isin(sales_packages_df['TICKET_ID'])]
sales_packages_df = sales_packages_df.groupby('EVENT_USAGE_DATE').size().reset_index(name='Sales_Packages_Count')
attendance_packages_df = attendance_packages_df.groupby('EVENT_DATE').size().reset_index(name='Attendance_Packages_Count')

In [12]:
packages_df = pd.merge(sales_packages_df, attendance_packages_df, how= 'inner', right_on='EVENT_DATE', left_on ='EVENT_USAGE_DATE')
packages_df = packages_df.drop(columns=['EVENT_DATE'])
packages_df


,EVENT_USAGE_DATE,Sales_Packages_Count,Attendance_Packages_Count
0,2024-04-02,2824,1329
1,2024-04-05,2733,773
2,2024-04-06,2689,712
3,2024-04-07,2844,1009
4,2024-04-16,2624,1273
...,...,...,...
68,2024-09-11,2811,945
69,2024-09-12,2725,1082
70,2024-09-13,2830,1577
71,2024-09-14,2672,1393


In [13]:
final_df = pd.merge(final_df, packages_df, how='inner', on='EVENT_USAGE_DATE')
final_df



,EVENT_USAGE_DATE,Sales_Singles_Count,Attendance_Singles_Count,Sales_Groups_Count,Attendance_Groups_Count,Sales_Packages_Count,Attendance_Packages_Count
0,2024-04-02,2750,1956,847,293,2824,1329
1,2024-04-05,590,485,686,359,2733,773
2,2024-04-06,637,510,354,254,2689,712
3,2024-04-07,621,560,446,388,2844,1009
4,2024-04-16,1160,1079,135,84,2624,1273
...,...,...,...,...,...,...,...
68,2024-09-11,729,627,977,534,2811,945
69,2024-09-12,979,767,1821,1013,2725,1082
70,2024-09-13,2729,2111,2484,1437,2830,1577
71,2024-09-14,3126,2492,2151,1255,2672,1393


In [14]:
final_df.to_csv('Sales+AttenadanceCounts24.csv')